In [ ]:
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype

def __get_cols_cleaning(p_pd_list:list) -> list:
    cols_cleaning = []
    for df in p_pd_list:
        #결측치 컬럼 조회
        tmp = df.isnull().sum()
        cols_cleaning += list(tmp[df.isnull().sum() > 0].index)
        
    cols_cleaning = list(set(cols_cleaning))
    print(f'cols_cleaning: {cols_cleaning}')
    return cols_cleaning

def add_cols_cleaning(p_train:pd.DataFrame, p_test:pd.DataFrame, p_ori_te:pd.DataFrame) -> None:
    pd_list = [p_train, p_test, p_ori_te]
    for col in __get_cols_cleaning(pd_list):
        for i, df in enumerate(pd_list, start=1):
            if is_numeric_dtype(df[col]): # 숫자형 결측치 처리
                new_col = col+'_cleaning'
                df[new_col] = df[col]   
                # random sampling
                random_sampling = (p_train[col].dropna().sample(df[col].isnull().sum()))
                # index 부여
                random_sampling.index = df[lambda x: x[col].isnull()].index 
                # NA imputation
                df.loc[df[col].isnull(), new_col] = random_sampling
        
                # 기존 결측치 컬럼 제거
                if i == len(pd_list):
                    for _df in pd_list:
                        _df.drop([col], axis=1, inplace=True)
      
          else: # 범주형 결측치 처리
            df[col].fillna(p_train[col].mode().values[0], inplace=True)
            
print(f'p_train: {p_train.isnull().sum().sum()} / p_test{p_test.isnull().sum().sum()} / p_ori_te: {p_ori_te.isnull().sum().sum()}')            

In [ ]:
add_cols_cleaning(train, test, ori_te)